# Unsupervised learning

Now that we have the data, let's figure out if we can pull out some features from it that can help us classify the types of data that it is.

In this notebook, we will:
* Use `umap` and `Kmeans` to find potential clusters for the listing data that can be used for isolating and modeling different chunks of the data.
* Use `PCA` to reduce all of the county and state level data into a few dimensions that represent the majority of the variance.

# Import packages.

In [ ]:
import pandas as pd
import numpy as np

# Dimensionality reduction: preprocessing.
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Dimensionality reduction: PCA.
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

# Clustering.
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Dimensionality reduction: UMAP.
import umap.umap_ as umap
from umap import utils
import altair as alt
alt.data_transformers.disable_max_rows()

# visualization.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# Progress tracking.
from tqdm import tqdm

# Show all columns
pd.set_option('display.max_columns', None)


# 0. Import Data and filter out nulls.
Read in the cleaned data from the earlier notebooks. We'll make a small tweak to the data in `airbnb_df` before proceeding with our project.

In [ ]:
# Read data
airbnb_df = pd.read_csv('CLEANED airbnb_data.csv').drop(columns=['Unnamed: 0'])
county_df = pd.read_csv('CLEANED county_data.csv').drop(columns=['Unnamed: 0'])
parks_df = pd.read_csv('CLEANED natl_state_park_by_us_state.csv')

In [ ]:
airbnb_df.head()

### 0.0 Drop listings with null values.

A decent chunk of the listings we scraped were unable to get any data on the listing's page. With those listings, we only have data on the search pages.

if we throw out all listings that don't have at least some data from the listing's page, we'll reduce the amount of data we need to impute. This should help increase the odds of finding latent, distinct clusters existing within the data when we go to cluster it.

* Asess `len()` of `airbnb_df`.
* Save copy of `airbnb_df` as `original` before dropping records.
* Drop all records where `n_guests` is null (proxy for not being able to access the listing's page).
* Asess `len()` of `airbnb_df` after dropping records.

In [ ]:
# Take a look at the results before scraping.
print('# of listings:',len(airbnb_df))

In [ ]:
# Copy the pre-filtered data for comparision after filtering.
original = airbnb_df.copy()

# Drop listings where n_guests is null. This is a good proxy for not being able to scrape any data
# on the listing's webpage.
airbnb_df.dropna(axis=0, subset=['n_guests'],inplace=True)

In [ ]:
# Take a look at the results after scraping.
print('# of listings:',len(airbnb_df))

### 0.I Visually assess filtered data.

Although we've dropped 45% of the data, we still have just over 100,000 records. Looking at the visual comparision of `original` and `airbnb_df` side by side, the filtered data seems to be drawing reasonably from the distribution without over indexing on listings from any one area of the country.

* Create subplot with `plt.subplots(1,2)`.
* On the first subplot, show `original` data, where each dot is a listing.
* On the second subplot, show `airbnb_df` data, where each dot is a listing.
* Render the plot with `plt.show`.

In [ ]:
# A quick side by side chart showing the data, before and after.
fig, axs = plt.subplots(1,2)

# Sizing.
fig.set_figheight(5)
fig.set_figwidth(15)

#plot 1:
axs[0].scatter(original['longitude'],original['latitude'],s=1,alpha=.05)
axs[0].set_title('Original Data ({} listings)'.format(len(original)))
axs[0].set_xlabel('longitude')
axs[0].set_ylabel('latitude')

#plot 2:
axs[1].scatter(airbnb_df['longitude'],airbnb_df['latitude'], s=1,alpha=.05)
axs[1].set_title('Filtered Data ({} listings)'.format(len(airbnb_df)))
axs[1].set_xlabel('longitude')
axs[1].set_ylabel('latitude')

# Show plot
plt.show()

# I. Clustering on listing attributes.

Using the features scraped off of AirBnB's website, we are going to see if we can draw out any structures of interest from the data. The goal would be to see if we could find either reduced dimensions features for supervised learning predictions, or clusters that could be used to partition data for different models.

### I.I Pre-process Data.

For our unsupervised learning, we'll need to change this data into a shape that can be accepted as an input for machine learning. First, we'll drop all columns that aren't numeric (or numeric columns that we don't think are predictors, like `postcode`, `latitude`, `longitude`). Then, we will normalize the data. While normalizing the data isn't technically required for the data to be in acceptable state for unsupervised machine learning, skipping this step can weaken the performance of the model. Scaling all data down to the same distribution makes it easier to pick out patterns, and not have the algorithm trip over features that have differences in their scale.

* Drop columns that won't be used for clustering.
* Use `StandardScaler` to normalize data.
* Use `SimpleImputer` to impute the median for missing values (there are still some of these in the records we have left).
* Return `x_normalized` for unsupervised learning.

In [ ]:
# Set up PCA model - airbnb.

# copy data.
pca_airbnb_df = airbnb_df.copy()

# columns that are not int/floats, or that deal with geography.
dropcols = ['listing_url','country','state','city','county','postcode','listing_title',
       'latitude','longitude','description','image_1','image_2','image_3','image_4','image_5']

# drop columns.
pca_airbnb_df.drop(columns=dropcols, inplace=True)

In [ ]:
# Split data. Remove listing id (just and ID) and price (leaks data to supervised learning model).
y = pca_airbnb_df[['listing_id','price']]
X = pca_airbnb_df.drop(columns=['listing_id','price'])

In [ ]:
X_normalized = StandardScaler().fit(X).transform(X)

imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_normalized = imp_median.fit_transform(X_normalized)

### I.III Visually assess normalized data.

We'll take a quick look at two variables, `price` and `n_amenities`, and see how normalization changed the relationship between the two. 

In [ ]:
# A quick side by side chart showing the data, before and after.
fig, axs = plt.subplots(1,2)

# Sizing.
fig.set_figheight(5)
fig.set_figwidth(15)

#plot 1:
axs[0].scatter(pca_airbnb_df['price'],pca_airbnb_df['n_amenities'],s=1,alpha=.05)
axs[0].set_title('Unnormalized Data')
axs[0].set_xlabel('Price')
axs[0].set_ylabel('# Amenities')

#plot 2:
axs[1].scatter(X_normalized[:,10],X_normalized[:,22], s=1,alpha=.05)
axs[1].set_title('Normalized Data')
axs[1].set_xlabel('Price')
axs[1].set_ylabel('# Amenities')

# Show plot
plt.show()

### I.IV PCA

Principal Component Analysis can help us try to find new features that reduce the variance of multiple other variables into fewer dimensions.

* Create a PCA model called `pca` that gets the 10 nearest components.
* Visually assess the first two dimensions.
* Plot all 10 components and their relationship between different variables.
* Plot the explained variance for each component.

In [ ]:
pca = PCA(n_components = 10, random_state=0).fit(X_normalized)

In [ ]:
pca_values = PCA(n_components = 10).fit_transform(X_normalized)

plt.scatter(pca_values[:,0], pca_values[:,1])
plt.title('First two dimensions: PCA')
plt.xlabel('2nd dimension')
plt.ylabel('1st dimension')
plt.show()

In [ ]:
def plot_pca(pca, top_k = 2):
    pca = PCA(n_components = 10).fit(X_normalized)
    fig, ax = plt.subplots(figsize=(18, 12))
    plt.imshow(pca.components_[0:top_k], interpolation = 'none', cmap = 'plasma', aspect='auto')
    labels= np.round(pca.components_[0:top_k],1)
    sns.heatmap(pca.components_[0:top_k], cmap="plasma", cbar=False, annot=labels, annot_kws={'fontsize': 8})
    feature_names=list(X.columns)
    plt.xticks(np.arange(-0., len(feature_names), 1) , feature_names, rotation = 90, fontsize=12)
    plt.yticks(np.arange(0., 10, 1), ['1st PC', '2nd PC','3rd PC','4th PC', '5th PC', 
                                     '6th PC','7th PC','8th PC','9th PC', '10th PC'],rotation=90, fontsize = 16)
    plt.colorbar()
    
    
plot_pca(pca, top_k=10)

In [ ]:
df = pd.DataFrame({'explained var ratio':pca.explained_variance_ratio_,
                   'PC':['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']})
sns.barplot(x = 'PC',y = "explained var ratio", data = df, color="c");
np.cumsum(df['explained var ratio'])

### Analysis of PCA on AirBnB listing data.

PCA did not do a strong job in explaining the variance of all the data in `airbnb_df`. The top 10 components explained a total of 54.5% of the variance in the data. I would have liked to see closer to 70-80% by the 5th component.

We weren't able to pull out reduced dimensions for features from `airbnb_df`. However, we might still be able to extract clusters using other techniques. Clustering will be important for supervised machine learning, as we can use the clusters to partition data into separate models and potentially increase the ability to predict a listings price.

### I.V Generating UMAP Embeddings

Uniform Manifold Approximation and Projection, or `umap`, can be used for non-linear dimensionality reduction. We might be able to identify some strong `Kmeans` clusters on top of this, but first we need to do a little hyperparameter tuning.

* Creating a list with each unique combination of the following metrics and neighbors: metrics: (`mahalanobis`, `wminkowski`, `cosine`, `correlation`), neighbors: (`100`, `300`, `500`).
* For each combination, extract the 2D embeddings for each model.
* Save the embedding locally in a file that will look like `listing_cosine_100.csv`.

In [ ]:
# Establish metrics and neighbors.
metrics = ['mahalanobis', 'wminkowski', 'cosine', 'correlation']
neighbors = [100, 300, 500]

# Empty list for model parameters.
model_parameters = []

# Append each unique metric-neighbor combination to model_parameters.
for metric in metrics:
    for neighbor in neighbors:
        model_parameters.append((metric, neighbor))

# Check outputs.
print(model_parameters)

In [ ]:
# Loop through each tuple in model_parameters.
for parameters in model_parameters:
    
    # Extract the metric and the neighbor values.
    metric = parameters[0]
    neighbors = parameters[1]
    
    # Extract embeddings for X_normalized using the metric and neighbors.
    embedding = umap.UMAP(n_neighbors=neighbors,
                          min_dist=0.0,
                          random_state=0,
                          metric=metric,
                          verbose=True
                    ).fit_transform(
                          X_normalized
                    )
    
    # Save the outputs of the model to prevent overloading computer memory.
    np.savetxt('listing_{}_{}.csv'.format(metric, neighbors),embedding, delimiter=',')

### I.VI UMAP Performance

We can now to start to assess how well these models performed in creating separated clusters. This can give us a sense of which model(s) had the best performance breaking apart the `umap` embeddings into distinct groups.

* Read in all of the csv files created by the different `umap` model variants.
* Test `KMeans` clustering models on each variant. Test between 2 and 11 clusters.
* Select the cluster value with the highest silhouette score for that model variant (a measurement of intra-cluster cohesiveness and inter-cluster separation).
* Visualize optimal clusters for that `umap` model in a chart, showing the silhouette score.

In [ ]:
# Read in all embeddings that were generated in the prior step.
mahalanobis_100 = pd.read_csv('listing_mahalanobis_100.csv',header=None,names=['x','y'])
mahalanobis_300 = pd.read_csv('listing_mahalanobis_300.csv',header=None,names=['x','y'])
mahalanobis_500 = pd.read_csv('listing_mahalanobis_500.csv',header=None,names=['x','y'])
wminkowski_100 = pd.read_csv('listing_wminkowski_100.csv',header=None,names=['x','y'])
wminkowski_300 = pd.read_csv('listing_wminkowski_300.csv',header=None,names=['x','y'])
wminkowski_500 = pd.read_csv('listing_wminkowski_500.csv',header=None,names=['x','y'])
cosine_100 = pd.read_csv('listing_cosine_100.csv',header=None,names=['x','y'])
cosine_300 = pd.read_csv('listing_cosine_300.csv',header=None,names=['x','y'])
cosine_500 = pd.read_csv('listing_cosine_500.csv',header=None,names=['x','y'])
correlation_100 = pd.read_csv('listing_correlation_100.csv',header=None,names=['x','y'])
correlation_300 = pd.read_csv('listing_correlation_300.csv',header=None,names=['x','y'])
correlation_500 = pd.read_csv('listing_correlation_500.csv',header=None,names=['x','y'])

In [ ]:
def gen_chart(df,title,optimal_clusters=None,score=None):
    
    if optimal_clusters==None:
        # Empty list for scores.
        silhouette_coefficients = []

        # Loop through cluster values between 2 and 10.
        for k in tqdm(range(2, 11)):

            # Create Kmeans models.
            kmeans = KMeans(
                init="random",
                n_clusters=k,
                n_init=10,
                max_iter=300,
                random_state=0
            )

            # Fit model.
            kmeans.fit(df[['x','y']])

            # Get silhouette score, append to list.
            score = silhouette_score(df[['x','y']], kmeans.labels_)
            silhouette_coefficients.append(score)

        # Identify optimal clusters for model variant.
        max_value = max(silhouette_coefficients)
        max_index = [index for index, item in enumerate(silhouette_coefficients) if item == max_value][0]
        optimal_clusters = range(2,11)[max_index]
    
    # Generate optimal clusters Kmeans.
    kmeans = KMeans(
            init="random",
            n_clusters=optimal_clusters,
            n_init=10,
            max_iter=300,
            random_state=0
        )
    
    # Fit model.
    kmeans.fit(df[['x','y']])
    
    # Save label to embeddings DataFrame.
    df['label'] = kmeans.labels_.astype('str')
    
    # Create chart to show clustering and silhouette score.
    chart = alt.Chart(df).mark_point(
        filled=True,
        size=10
    ).encode(
        x='x',
        y='y',
        color='label:N'
    ).properties(
        title=(title, 'Optimal Clusters={},Silhouette Score={}'.format(optimal_clusters, score)),
        width=200,
        height=250
    )
    
    # Return chart.
    return chart

In [ ]:
# Generate charts for Mahalanobis variants.
m100 = gen_chart(mahalanobis_100,'Metric: Mahalanobis; Neighbors: 100',optimal_clusters=None)
m300 = gen_chart(mahalanobis_300,'Metric: Mahalanobis; Neighbors: 300',optimal_clusters=None)
m500 = gen_chart(mahalanobis_500,'Metric: Mahalanobis; Neighbors: 500',optimal_clusters=None)

# Generate charts for Wminkowski variants.
wm100 = gen_chart(wminkowski_100,'Metric: Wminkowski; Neighbors: 100',optimal_clusters=None)
wm300 = gen_chart(wminkowski_300,'Metric: Wminkowski; Neighbors: 300',optimal_clusters=None)
wm500 = gen_chart(wminkowski_500,'Metric: Wminkowski; Neighbors: 500',optimal_clusters=None)

# Generate charts for Cosine variants.
cos100 = gen_chart(cosine_100,'Metric: Cosine; Neighbors: 100',optimal_clusters=None)
cos300 = gen_chart(cosine_300,'Metric: Cosine; Neighbors: 300',optimal_clusters=None)
cos500 = gen_chart(cosine_500,'Metric: Cosine; Neighbors: 500',optimal_clusters=None)

# Generate charts for Correlation variants.
cor100 = gen_chart(correlation_100,'Metric: Correlation; Neighbors: 100',optimal_clusters=None)
cor300 = gen_chart(correlation_300,'Metric: Correlation; Neighbors: 300',optimal_clusters=None)
cor500 = gen_chart(correlation_500,'Metric: Correlation; Neighbors: 500',optimal_clusters=None)

# Create concatenated chart.
alt.vconcat(
    alt.hconcat(m100,m300,m500),
    alt.hconcat(wm100,wm300,wm500),
    alt.hconcat(cos100,cos300,cos500),
    alt.hconcat(cor100,cor300,cor500)
)

### Analysis of UMAP Performance

The models performed between 0.405 and 0.515. Since the range of a silhouette score can be between -1 and 1, these clusters are information but perhaps not definitive clusters. However, there might be something here that can be used for supervised learning.

* **Top Mahalanobis model:** 100 neighbors, 2 clusters _(silhouette score: .453)_
* **Top Wminkowski model:** 500 neighbors, 7 clusters _(silhouette score: .405)_
* **Top Cosine model:** 100 neighbors, 2 clusters _(silhouette score: .499)_
* **Top Correlation model:** 100 neighbors, 2 clusters _(silhouette score: .515)_
* **Top Overall Model:** Correlation, 100 neighbors, 2 clusters _(silhouette score: .515)_

### I.VII Save top model for each metric to data.

We'll take the neighbor value for each metric, cluster it by the `Kmeans` with the highest silhouette score, and save it as a column in the `airbnb_df` DataFrame.

* Function for returning labels for specific embedding / k clusters combination.
* Save top model for each metric's labels as a new column in the DataFrame.

In [ ]:
def add_cluster_labels(df, k):
    
    # Create kmeans algorithm.
    kmeans = KMeans(
            init="random",
            n_clusters=k,
            n_init=10,
            max_iter=300,
            random_state=0
        )

    # Fit to dataset.
    kmeans.fit(df[['x','y']])
    
    # Return labels.
    return kmeans.labels_.astype('str')
    

In [ ]:
# Save labels to new columns of DataFrames.
airbnb_df['mahalanobis_100'] = add_cluster_labels(df=mahalanobis_100,k=2)
airbnb_df['wminkowski_500'] = add_cluster_labels(df=wminkowski_500,k=7)
airbnb_df['cos100'] = add_cluster_labels(df=cosine_100,k=2)
airbnb_df['cor100'] = add_cluster_labels(df=correlation_100,k=2)

### I.VIII Visualize elements with highest correlation to price in each cluster for each model.

We want to get some intuitive sense of what each cluster in each model might be picking up on, and how those trends might correlate to the price of the listing.

* Plot each of the top models, and how each variable correlates to price.
* Analyze trends that exist within each cluster of each model.

In [ ]:
def plot_umap(col, airbnb_df=airbnb_df):
    
    # Create copy to prevent mutating the original DataFrame.
    df = airbnb_df.copy()
    
    # columns that are not int/floats, or that deal with geography.
    dropcols = ['listing_url','country','state','city','county','postcode','listing_title',
       'latitude','longitude','description','image_1','image_2','image_3','image_4','image_5',
               'listing_id','mahalanobis_100','wminkowski_500','cos100','cor100']
    
    # Exclude the group column from the drop list.
    dropcols.remove(col)
    
    # drop columns.
    df.drop(columns=dropcols, inplace=True)
    
    # Empty variable to build chart on.
    concat_chart = None
    
    # Loop through each cluster.
    for val in sorted(df[col].unique()):
        
        # Create a copy to prevent mutating the DataFrame `df`.
        frame = df.copy()
        
        # Filter for only that cluster.
        frame = frame[frame[col] == val]
        
        # Get a correlation matrix, sorted by variables that correlate with price.
        corr = frame.corr().sort_values(by='price')
        corr.drop(index='price',inplace=True)
        
        # Create a chart showing the correlations with price.
        chart = alt.Chart(corr.reset_index()).mark_bar().encode(
            x=alt.X('index',sort='y', title='Feature'),
            y=alt.Y('price',title='Correlation with Price')
        ).properties(
            title='Price Correlations for {}, cluster {}'.format(col, val))
        
        # If concat_chart is None.
        if concat_chart == None:
            # Replace it with the chart.
            concat_chart = chart
        # If there is already a chart,
        else:
            # Vertically concatenate the new chart to the old chart.
            concat_chart = concat_chart & chart
    
    # Return concetenated chart.
    return concat_chart.resolve_scale(
        x='independent'
    )
    


In [ ]:
plot_umap('mahalanobis_100')

### Analysis: metric=mahalanobis, neighbors=100

**cluster 0**
* Most positive correlating variables: `n_baths`, `bed_king`, `n_bedrooms`.
* Most negative correlating variables: `private_room`, `n_reviews`, `amenities_not_included`.

**cluster 1**
* Most positive correlating variables: `n_baths`, `n_bedrooms`, `n_beds`. 
* Most negative correlating variables: `is_superhost`, `n_reviews`, `rating_value`.

In [ ]:
plot_umap('wminkowski_500')

### Analysis: metric=Wminkowski, neighbors=500

**cluster 0**
* Most positive correlating variables: `n_baths`, `entire_home`, `bed_king`.
* Most negative correlating variables: `n_ratings`, `private_room`, `rating_value`.

**cluster 1**
* Most positive correlating variables: `bed_king`, `bed_single`, `bed_queen`.
* Most negative correlating variables: `rating_value`, `n_ratings`, `private_room`.

**cluster 2**
* Most positive correlating variables: `n_baths`, `n_beds`, `n_guests`.
* Most negative correlating variables: `amenities_entertainment`, `n_ratings`, `rating_value`.

**cluster 3**
* Most positive correlating variables: `n_guests`, `n_beds`, `bed_king`.
* Most negative correlating variables: `rating_value`, `n_reviews`, `perc_discount`.

**cluster 4**
* Most positive correlating variables: `n_guests`, `n_beds`, `bed_king`.
* Most negative correlating variables: `private_room`, `amenities_safety`, `amenities_bathroom`.

**cluster 5**
* Most positive correlating variables: `n_baths`, `n_beds`, `bed_king`.
* Most negative correlating variables: `private_room`, `n_reviews`, `is_superhost`.

**cluster 6**
* Most positive correlating variables: `n_baths`, `amenities_entertainment`, `n_bedrooms`.
* Most negative correlating variables: `private_room`, `amenities_not_included`, `n_reviews`.

In [ ]:
plot_umap('cos100')

### Analysis: metric=Cosine, neighbors=100

**cluster 0**
* Most positive correlating variables: `n_beds`, `bed_king`, `n_bedrooms`.
* Most negative correlating variables: `amenities_kitchen_dining`, `amenities_safety`, `amenities_bathroom`.

**cluster 1**
* Most positive correlating variables: `n_baths`, `bed_king`, `n_bedrooms`.
* Most negative correlating variables: `n_reviews`, `is_superhost`, `rating_value`.

In [ ]:
plot_umap('cor100')

### Analysis: metric=Correlation, neighbors=100

**cluster 0**
* Most positive correlating variables: `n_baths`, `bed_king`, `n_bedrooms`.
* Most negative correlating variables: `n_reviews`, `is_superhost`, `rating_value`.

**cluster 1**
* Most positive correlating variables: `n_guests`, `n_beds`, `bed_king`.
* Most negative correlating variables: `amenities_kitchen_dining`, `amentities_safety`, `amenities_bathroom`.

### Analysis: Insights from looking at cluster-specific pricing correlations.

* Variables that indicate the size of the listing (`n_baths`, `n_guests`, `n_bedrooms`) consistently have positive correlations with price.
* Variables that indicate a smaller loding (`is_private_room`, `amenities_not_included`) consistently have negative correlations with price.
* In some clusters, `bed_king` is picked up as a variable that correlates with price. Perhaps this suggests these clusters are picking up on more larger listings with big beds.
* Suprisingly, the number of reviews and ratings can sometimes negative correlate with `price`. However, we should exercise caution. Median imputing affect the trends here. There might also be an explanation that much more expensive listings (1K+/night) might have less reviews than cheaper listings (100/night).

One thing I might need to watch out for in the supervised learning model is outliers in price. There might be some outliers that affect the way the model comes together. Because these clusters did not look at price data I'm not too worried about reassessing the unsupervised machine learning models, but it might be something to consider in data cleaning for the unsupervised models.

# I.IX Sensitivity Analysis

In [ ]:
correlation_x, correlation_y = [1,2,3], [0.515,0.461,0.471]
cosine_x, cosine_y = [1,2,3], [.499,0.487,0.472]
mahalanobis_x, mahalanobis_y = [1,2,3], [0.453,0.401,0.411]
wminkowski_x, wminkowski_y = [1,2,3], [0.401,0.398,0.405]


plt.figure(figsize=(15,8))

plt.plot(correlation_x, correlation_y,label='correlation')
plt.plot(cosine_x, cosine_y,label='cosine')
plt.plot(mahalanobis_x, mahalanobis_y,label='mahalanobis')
plt.plot(wminkowski_x, wminkowski_y,label='wminkowski')

plt.xlabel('Neighbors')
plt.ylabel('Silhouette Score')
plt.xticks([1,2,3],['100','300','500'])

plt.title('Sensitivity Score')
plt.legend()

plt.show()




# Part II: Clustering on Geographic attributes

Now, let's separately handle looking at dimensionality reduction for the geographic attributes. Our goal would be to see if we can extract some `pca` components that reduce some of the geography.

### II.I: Create `geo_df`

Let's get a DataFrame stood up with one record for each listing, and all of the geographic data we have on the listing on the area the listing is located.

* Extract all geographic data out of `airbnb_df` into `airbnb_df_geo`.
* Join `airbnb_df_geo` with `county_geo` and `parks_df`.

In [ ]:
# Get geo-data in airbnb_df.
airbnb_df_geo = airbnb_df[['listing_id','country','state','city','county','postcode']]

# Drop FIPS code.
county_geo = county_df.drop(columns=['FIPS'])

# A quick cleaner function to help join the datasets.
def remove_county(x):
    try:
        return x.replace(' County','')
    except:
        return x


# Clean county data by removing ' County' from name.
airbnb_df_geo['county'] = airbnb_df_geo['county'].apply(remove_county)

# Rename county_name to county. This approach is a little lazy.
county_geo['county'] = county_geo['county_name']

# Merge data.
geo_df = airbnb_df_geo.merge(county_geo,how='left',on=['state','county'])
geo_df = geo_df.merge(parks_df,how='left',on='state')

# Look at first five records.
geo_df.head()

### II.II Prepare data for PCA.

Similar to what we did with `airbnb_df`, let's prepare `geo_df` for a Principal Components Analysis.

* Save numerical values of interest in X.
* Normalize data with `StandardScaler().
* Impute missing data with median.
* Replace any `inf` values with 0.

In [ ]:
Y = geo_df['listing_id']

X = geo_df[['rural-urban-continuum-code','perc-college-educated-2016-2020','perc_unemployed_2021',
           'median_hh_income','perc_state_median_hh_income','perc_people_poverty_2020',
           'perc_children_poverty_2020','count_state_park','count_national_park','percent_park']]

X.head()

In [ ]:
X_normalized = StandardScaler().fit(X).transform(X)

imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_normalized = imp_median.fit_transform(X_normalized)

X_normalized[X_normalized == -np.inf] = 0


### II.III PCA with Geographic Data.

Let's see if we have better luck with this smaller dataset finding good principal components.

* Create PCA model `pca` with 10 components fit on `X_normalized`.
* Visualize the first 2 dimensions.
* Visualize the relationship between each component and the variables.
* Visualize the explained variance ratio.

In [ ]:
pca = PCA(n_components = 10, random_state=0).fit(X_normalized)
pca_values = PCA(n_components = 10).fit_transform(X_normalized)

plt.scatter(pca_values[:,0], pca_values[:,1])
plt.title('First two dimensions: PCA')
plt.xlabel('2nd dimension')
plt.ylabel('1st dimension')
plt.show()

In [ ]:
plot_pca(pca, top_k=10)

In [ ]:
df = pd.DataFrame({'explained var ratio':pca.explained_variance_ratio_,
                   'PC':['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']})
sns.barplot(x = 'PC',y = "explained var ratio", data = df, color="c");
np.cumsum(df['explained var ratio'])

### Analysis: Principal components.

`pca` did a fairly good job here in dimensionality reduction. The first 3 components explain 86.4% of the variance. Additionally, the factors that relate to each component tell a human interpretable story about what is captured in each component:

* **1st component (43.19% of variance):** Positive correlation with `perc_people_in_poverty_2020`, `perc_children_in_poverty_2020`, and `rural_urban_continuum_code` (higher score here implies a more rural area); negative correlation with `perc-college-educated-2016-2022 `, `median_hh_income`, and `perc_median_state_household_income`. Listings with a high score on the first component are more rural, less wealthy, and less educated.
* **2nd component (34.19% of variance:** Positive correlation with `count_state_park`, `count_national_park`, `percent_park`, and `perc_unemployed_2021`; slight negative correlation with `rural_urban_continuum_code` and `perc_college_educated_2016-2022`. Listings with a high score on the second component are slightly more urban, slightly less educated and employed, and (most importantly) in states with a large parks presence. These are likely cities and counties with sub-urban communities near the state's median income.
* **3rd component (9.02%):** Positive correlation with `perc_unemployed_2021`; negative correlation with `rural-urban-continuum-code`, `count_national_park`, `perc_parks`. Listings with a high score on the third component are more urban, have higher unemployment, and have less national parks presence in the state. This likely captures urban communities in states with less parks (such as large cities in New Mexico, Nevada, or Idaho).

We don't necessarily need to go through the exercise of using `umap` to try and find clusters. These reduced dimensions (in combination with the `umap` clusters derived from the listings data) should sufficiently enrich the data to try and build an interesting supervised learning model on predicting price.

In [ ]:
airbnb_df['pca_1st_component'] = pca_values[:,0] 
airbnb_df['pca_2nd_component'] = pca_values[:,1]
airbnb_df['pca_3rd_component'] = pca_values[:,2]
airbnb_df['pca_4th_component'] = pca_values[:,3]
airbnb_df['pca_5th_component'] = pca_values[:,4]

# III. Save Outputs.

Let's save the outputs of this analysis to carry forward to the Supervised Machine Learning notebook (3.2).

### III.I Save data.

In [ ]:
airbnb_df.head()

In [ ]:
airbnb_df.to_csv('UNSUPERVISED_LEARNING airbnb_data.csv')